In [2]:
import pandas as pd
import numpy as np

### Loading and cleaning data:

In [18]:
dw = (pd.read_csv("../data/initial.csv", index_col=0)
      .rename(columns={"mode": "Condition", "hotspot": "Hotspot"})
      .assign(Condition = lambda df: df['Condition'].map(lambda c: "NG" if c == "N" else "G"))
      #.assign(Condition = lambda df: pd.Categorical(df['Condition']),
      #       Hotspot = lambda df: pd.Categorical(df['Hotspot']))
     )
dw

,Hotspot,Condition,P1,P2,P3,P4
1,1,NG,120,120,120,82
2,10,G,20,13,2,14
3,2,NG,120,120,73,120
4,4,G,53,20,24,15
5,6,NG,120,120,120,70
6,7,G,41,40,20,28
7,8,NG,120,120,120,120
8,8,G,11,14,24,9
9,9,NG,120,120,13,4
10,3,G,13,1,8,1


### Moving to long format:

In [36]:
dl = (pd.melt(dw.reset_index(), id_vars=['index', 'Hotspot', 'Condition'], var_name='Participant')
      .rename(columns={"index": "Trial", "value": "Time"})
      .sort_values(['Participant', 'Trial', 'Condition'])
      .reindex(columns=["Participant", "Trial", "Hotspot", "Condition", "Time"])
     )
dl

,Participant,Trial,Hotspot,Condition,Time
0,P1,1,1,NG,120
1,P1,2,10,G,20
2,P1,3,2,NG,120
3,P1,4,4,G,53
4,P1,5,6,NG,120
...,...,...,...,...,...
75,P4,16,6,G,15
76,P4,17,4,NG,120
77,P4,18,9,G,9
78,P4,19,3,NG,118


### Rearranging data for Wilcoxon:

In [37]:
dl_wil = dl.sort_values(['Participant', 'Hotspot', 'Condition'])
dl_wil

,Participant,Trial,Hotspot,Condition,Time
19,P1,20,1,G,31
0,P1,1,1,NG,120
13,P1,14,2,G,14
2,P1,3,2,NG,120
9,P1,10,3,G,13
...,...,...,...,...,...
66,P4,7,8,NG,120
77,P4,18,9,G,9
68,P4,9,9,NG,4
61,P4,2,10,G,14


In [48]:
from scipy.stats import wilcoxon

V, p = wilcoxon(dl_wil.query("Condition == 'G'").loc[:,"Time"], dl_wil.query("Condition == 'NG'").loc[:,"Time"], alternative="two-sided")
print(f" V statistic: {V}\n p-value: {p}")

 V statistic: 5.0
 p-value: 5.1782619495255144e-08
